In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Model
import numpy as np

# Path to Food-11 dataset
data_dir =r"C:\Users\omcha\Machine learning\Task5\archive"
# Data preprocessing
data_gen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,
    rotation_range=20,
    zoom_range=0.2,
    horizontal_flip=True
)

# Training and validation data
train_data = data_gen.flow_from_directory(
    f"{data_dir}/training",
    target_size=(224, 224),
    batch_size=32,
    class_mode="categorical"
)

val_data = data_gen.flow_from_directory(
    f"{data_dir}/validation",
    target_size=(224, 224),
    batch_size=32,
    class_mode="categorical"
)

# Load pre-trained MobileNetV2
base_model = MobileNetV2(weights="imagenet", include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False  # Freeze layers

# Add custom classification layers
x = GlobalAveragePooling2D()(base_model.output)
x = Dropout(0.5)(x)
output = Dense(11, activation="softmax")(x)

model = Model(inputs=base_model.input, outputs=output)

# Compile the model
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

# Train the model
history = model.fit(
    train_data,
    validation_data=val_data,
    epochs=10,
    batch_size=32
)

# Save the model
model.save("food11_classifier.h5")


In [ ]:
calorie_lookup = {
    "bread": 265, "dairy products": 150, "desserts": 450, "eggs": 155,
    "fried food": 312, "meat": 250, "noodles/pasta": 350,
    "rice": 200, "seafood": 150, "soup": 75, "vegetables/fruits": 50
}


In [ ]:
def estimate_calories(predicted_class):
    return calorie_lookup.get(predicted_class, "Unknown")


In [ ]:
# Load an image
image = tf.keras.utils.load_img("path/to/test_image.jpg", target_size=(224, 224))
image_array = np.expand_dims(tf.keras.utils.img_to_array(image) / 255.0, axis=0)

# Make predictions
predictions = model.predict(image_array)
predicted_class_index = np.argmax(predictions)
predicted_class = train_data.class_indices
class_name = list(predicted_class.keys())[predicted_class_index]

# Get calorie estimate
calories = estimate_calories(class_name)
print(f"Food Item: {class_name}, Estimated Calories: {calories} kcal")
